### Results: Decision + Voting

In [ ]:
dataset_type = "pathways"

def merge_dataframes(df1, df2):
    import pandas as pd
    # Concatenate the dataframes along the index
    merged_df = pd.concat([df1, df2], axis=0)
    # Group by the index and aggregate the values
    merged_df = merged_df.groupby(merged_df.index).first()
    return merged_df

import os
import pandas as pd
input_folder = "results"

for dirpath, dirnames, filenames in os.walk(input_folder):
    for file in filenames:
        if file.endswith(".xlsx") and dataset_type in file:
            print(file)
            file_path = os.path.join(input_folder, file)
            merged = pd.read_excel(file_path).set_index("run")

for dirpath, dirnames, filenames in os.walk(input_folder):
    for file in filenames:
        if file.endswith(".xlsx") and dataset_type in file:
            #print(file)
            file_path = os.path.join(input_folder, file)
            file_df = pd.read_excel(file_path).set_index("run")
            merged = merge_dataframes(merged, file_df)
display(merged.head())
#print("merged shape: ", merged.shape)

metadata = pd.read_csv("metadata/metadata.csv")
#display(metadata.head())
#print("metadata shape: ", metadata.shape)
metadata_samples = metadata['run'].tolist()
available_samples = list(merged.index)
metadata_samples = [sample for sample in metadata_samples if sample in available_samples]
#print(len(metadata_samples))
metadata = metadata.set_index("run")
metadata = metadata.loc[metadata_samples]
#display(metadata.head())
merged = merge_dataframes(metadata, merged)
                
display(merged.head())
merged.to_excel("merged_predictions.xlsx")

pathways.csv-predictions-general-pathways.xlsx


,TrueLabel,TrueLabel_Binary,Prediction: General Model,Probability
run,,,,
ERR2608606,TD,1,TD,0.52
ERR2608618,TD,1,ASD,0.41
ERR2608632,ASD,0,ASD,0.30
ERR2608634,ASD,0,ASD,0.47
ERR2608636,ASD,0,ASD,0.21


,bioProject,locationCountry,locationContinent,diagnosis,instrument,TrueLabel,TrueLabel_Binary,Prediction: General Model,Probability
run,,,,,,,,,
ERR2608606,PRJEB23052,China,Asia,TD,Illumina HiSeq 4000,TD,1.0,TD,0.52
ERR2608618,PRJEB23052,China,Asia,TD,Illumina HiSeq 4000,TD,1.0,ASD,0.41
ERR2608632,PRJEB23052,China,Asia,ASD,Illumina HiSeq 4000,ASD,0.0,ASD,0.30
ERR2608634,PRJEB23052,China,Asia,ASD,Illumina HiSeq 4000,ASD,0.0,ASD,0.47
ERR2608636,PRJEB23052,China,Asia,ASD,Illumina HiSeq 4000,ASD,0.0,ASD,0.21


In [ ]:
merged.to_excel("PREDICTIONS.xlsx")

In [5]:
def model_performance(df):
    from sklearn.metrics import (
        confusion_matrix, ConfusionMatrixDisplay,
        roc_auc_score, roc_curve,
        accuracy_score, precision_score, recall_score, f1_score
    )
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import LabelBinarizer

    # Extract from DataFrame
    y_test = df["diagnosis"]
    y_pred = df["Probability"]
    y_prob = df["highest_probability"]

    # Label encoding (binary) for AUC and consistency
    lb = LabelBinarizer()
    y_test_bin = lb.fit_transform(y_test).ravel()
    classes = lb.classes_

    print(f"Label encoding: {classes[0]} -> 0, {classes[1]} -> 1")

    # AUC and ROC
    if len(classes) == 2:
        auc_score = roc_auc_score(y_test_bin, y_prob)
        print(f"AUC: {auc_score:.2f}")

        fpr, tpr, _ = roc_curve(y_test_bin, y_prob)
        plt.figure()
        plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()
    else:
        print("AUC calculation for multiclass requires additional setup.")

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred, labels=classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.grid(False)
    plt.show()

    # Classification metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label=classes[1])
    recall = recall_score(y_test, y_pred, pos_label=classes[1])
    f1 = f1_score(y_test, y_pred, pos_label=classes[1])

    print(f"Accuracy : {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall   : {recall:.2f}")
    print(f"F1 Score : {f1:.2f}")

model_performance(merged)


KeyError: 'highest_probability'